In [1]:
from pathlib import Path
from dotenv import dotenv_values, load_dotenv


import coref_ds
from coref_ds.text import Text
from coref_ds.mmax.mmax_doc import MmaxDoc
from coref_ds.tei.tei_doc import TEIDocument

from plotkarzyna.model import get_model, predict

try:
    local_config = dotenv_values(".env")
except FileNotFoundError:
    local_config = dict()
load_dotenv()

[nltk_data] Downloading package punkt to /home/ksaputa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
n_samples = 100_000_000
test_paths = list(
    (Path(local_config['PCC_ROOT']) / 'test').iterdir()
    )[:n_samples]

In [3]:
tei_docs = []
for p in test_paths:
    try: 
        m = TEIDocument(p)
        tei_docs.append(m)
    except Exception as e:
        print(e)

In [4]:
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large-2/checkpoint-6240'
tokenizer, model = get_model(checkpoint_path)

In [5]:
from plotkarzyna.model import chunkize

In [6]:
chunks = chunkize(tei_docs[0].text.segments, tokenizer)

[32, 214, 211, 120, 114, 47, 91, 213, 93, 57, 1, 1, 73, 100, 74, 36, 291, 62] [391]


In [7]:
chunks

[['Nie</w>',
  'zawsze</w>',
  'jednak</w>',
  'to</w>',
  'wystarcza</w>',
  '.</w>',
  'Zmiany</w>',
  'w</w>',
  'środowisku</w>',
  ',</w>',
  'a</w>',
  'także</w>',
  'postęp</w>',
  'w</w>',
  'rolnictwie</w>',
  'i</w>',
  'nieustan',
  'ne</w>',
  'powtarz',
  'anie</w>',
  'błędów</w>',
  'Zachodu</w>',
  ',</w>',
  'gdzie</w>',
  'obecnie</w>',
  'na</w>',
  'od',
  'tworzenie</w>',
  'eko',
  'systemów</w>',
  'rolniczych</w>',
  'wydaje</w>',
  'się</w>',
  'ogromne</w>',
  'sumy</w>',
  ',</w>',
  'sprawiają</w>',
  ',</w>',
  'że</w>',
  'chwa',
  'sty</w>',
  'giną</w>',
  'coraz</w>',
  'szybciej</w>',
  '.</w>',
  'Czasem</w>',
  'jest</w>',
  'to</w>',
  'spowodowane</w>',
  'tym</w>',
  ',</w>',
  'że</w>',
  'niektóre</w>',
  'gatunki</w>',
  'roślin</w>',
  'wycof',
  'uje</w>',
  'się</w>',
  'z</w>',
  'uprawy</w>',
  '–</w>',
  'coraz</w>',
  'rzadziej</w>',
  'uprawia</w>',
  'się</w>',
  'len</w>',
  'albo</w>',
  'gry',
  'kę</w>',
  ',</w>',
  'nie</w>',
  

In [8]:
chunks

[['Nie</w>',
  'zawsze</w>',
  'jednak</w>',
  'to</w>',
  'wystarcza</w>',
  '.</w>',
  'Zmiany</w>',
  'w</w>',
  'środowisku</w>',
  ',</w>',
  'a</w>',
  'także</w>',
  'postęp</w>',
  'w</w>',
  'rolnictwie</w>',
  'i</w>',
  'nieustan',
  'ne</w>',
  'powtarz',
  'anie</w>',
  'błędów</w>',
  'Zachodu</w>',
  ',</w>',
  'gdzie</w>',
  'obecnie</w>',
  'na</w>',
  'od',
  'tworzenie</w>',
  'eko',
  'systemów</w>',
  'rolniczych</w>',
  'wydaje</w>',
  'się</w>',
  'ogromne</w>',
  'sumy</w>',
  ',</w>',
  'sprawiają</w>',
  ',</w>',
  'że</w>',
  'chwa',
  'sty</w>',
  'giną</w>',
  'coraz</w>',
  'szybciej</w>',
  '.</w>',
  'Czasem</w>',
  'jest</w>',
  'to</w>',
  'spowodowane</w>',
  'tym</w>',
  ',</w>',
  'że</w>',
  'niektóre</w>',
  'gatunki</w>',
  'roślin</w>',
  'wycof',
  'uje</w>',
  'się</w>',
  'z</w>',
  'uprawy</w>',
  '–</w>',
  'coraz</w>',
  'rzadziej</w>',
  'uprawia</w>',
  'się</w>',
  'len</w>',
  'albo</w>',
  'gry',
  'kę</w>',
  ',</w>',
  'nie</w>',
  

In [9]:
chunks[0][0]

'Nie</w>'

In [10]:
len(chunks[0])

391

In [11]:
tokenizer(chunks[0], is_split_into_words=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (1689 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [0, 2351, 1841, 1888, 1019, 1922, 3589, 1841, 1888, 1019, 1922, 2401, 1841, 1888, 1019, 1922, 2063, 1841, 1888, 1019, 1922, 24312, 1841, 1888, 1019, 1922, 1899, 1841, 1888, 1019, 1922, 14918, 1841, 1888, 1019, 1922, 1019, 1841, 1888, 1019, 1922, 12325, 1841, 1888, 1019, 1922, 1947, 1841, 1888, 1019, 1922, 1011, 1841, 1888, 1019, 1922, 2413, 1841, 1888, 1019, 1922, 24901, 1841, 1888, 1019, 1922, 1019, 1841, 1888, 1019, 1922, 24659, 1841, 1888, 1019, 1922, 1009, 1841, 1888, 1019, 1922, 46905, 1016, 2079, 1841, 1888, 1019, 1922, 2080, 5977, 2903, 1841, 1888, 1019, 1922, 12277, 1841, 1888, 1019, 1922, 20332, 1841, 1888, 1019, 1922, 1947, 1841, 1888, 1019, 1922, 2860, 1841, 1888, 1019, 1922, 4227, 1841, 1888, 1019, 1922, 1998, 1841, 1888, 1019, 1922, 2173, 12130, 1841, 1888, 1019, 1922, 73, 2119, 15080, 1841, 1888, 1019, 1922, 21696, 1841, 1888, 1019, 1922, 5634, 1841, 1888, 1019, 1922, 2022, 1841, 1888, 1019, 1922, 9510, 1841, 1888, 1019, 1922, 15244, 1841, 1888, 1019, 1922, 

In [12]:
len(tokenizer.encode_plus(chunks[0], is_split_into_words=True).input_ids)

1689

In [13]:
preds = []
for ind, doc in enumerate(tei_docs):
    # try:
    pred = predict(doc.text.segments, model, tokenizer)
    preds.append(pred)
    # except Exception as e:
    #     print(e)
    #     print(ind, doc.doc_path)
    # else:
    #     pass

[32, 214, 211, 120, 114, 47, 91, 213, 93, 57, 1, 1, 73, 100, 74, 36, 291, 62] [391]
chunks: [['Nie</w>', 'zawsze</w>', 'jednak</w>', 'to</w>', 'wystarcza</w>', '.</w>', 'Zmiany</w>', 'w</w>', 'środowisku</w>', ',</w>', 'a</w>', 'także</w>', 'postęp</w>', 'w</w>', 'rolnictwie</w>', 'i</w>', 'nieustan', 'ne</w>', 'powtarz', 'anie</w>', 'błędów</w>', 'Zachodu</w>', ',</w>', 'gdzie</w>', 'obecnie</w>', 'na</w>', 'od', 'tworzenie</w>', 'eko', 'systemów</w>', 'rolniczych</w>', 'wydaje</w>', 'się</w>', 'ogromne</w>', 'sumy</w>', ',</w>', 'sprawiają</w>', ',</w>', 'że</w>', 'chwa', 'sty</w>', 'giną</w>', 'coraz</w>', 'szybciej</w>', '.</w>', 'Czasem</w>', 'jest</w>', 'to</w>', 'spowodowane</w>', 'tym</w>', ',</w>', 'że</w>', 'niektóre</w>', 'gatunki</w>', 'roślin</w>', 'wycof', 'uje</w>', 'się</w>', 'z</w>', 'uprawy</w>', '–</w>', 'coraz</w>', 'rzadziej</w>', 'uprawia</w>', 'się</w>', 'len</w>', 'albo</w>', 'gry', 'kę</w>', ',</w>', 'nie</w>', 'mówiąc</w>', 'o</w>', 'takich</w>', '“</w>', 'dzi

RuntimeError: The size of tensor a (1689) must match the size of tensor b (514) at non-singleton dimension 1